In [ ]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

In [ ]:
from AutoScreen.AndroidScreen import ADBPropertiesHelper, AndroidController, UINode

import cv2
import xml.etree.ElementTree as ET
from script.utils import add_text_with_transparent_bg, draw_bbox_multi, AndroidUIHelper
import os

In [ ]:
temp_data_base_dir = os.path.join("data", "temp")

In [ ]:
for _ in range(3):
    try:
        android_controller = AndroidController()
        screenshot_raw_path = android_controller.get_screenshot("wechat_moment", temp_data_base_dir)
        xml_raw_path = android_controller.get_xml("wechat_moment", temp_data_base_dir)
        logging.info(f"Successfully saved screenshot and ui xml for {android_controller.device}")
        break
    except RuntimeError as e:
        android_controller.reopen_app_by_going_to_homescreen()

In [ ]:
from IPython.display import display, Image
# display(Image('data/temp/wechat_moment.png'))
display(Image(screenshot_raw_path))

In [ ]:

tree = ET.parse(xml_raw_path)
root = tree.getroot()

In [ ]:
ui_node_list = []
for ele in tree.iter():
    if ele.attrib.get("clickable", "false") == "true":
    # if ele.attrib.get("clickable", "false") == "true" or ele.attrib.get("focusable", "false") == "true":
        print(ele, ele.attrib)        
        left, high, right, low = AndroidUIHelper.get_box_from_bounds_raw_str(bounds=ele.attrib.get("bounds"))
        name = ele.attrib.get("resource-id", "no-resource-id").split('/')[-1]
        if name is None:
            name = ele.attrib.get("content-desc", "no-content-desc")
        ui_node_list.append(UINode(name, ((left, high), (right, low)), ele.attrib))

In [ ]:
len(ui_node_list)

In [ ]:
ui_node_list

In [ ]:
draw_bbox_multi(screenshot_raw_path, temp_data_base_dir + "/with_box.png", ui_node_list)

In [ ]:
from IPython.display import display, Image
display(Image(temp_data_base_dir + "/with_box.png"))

In [ ]:
root.findall(".//*[@resource-id='com.tencent.mm:id/n9a']")

In [ ]:
tree.findall(".//*[@resource-id='com.tencent.mm:id/n9a']")

In [ ]:
moment_node_list = tree.findall(".//*[@resource-id='com.tencent.mm:id/n9a']") + tree.findall(".//*[@resource-id='com.tencent.mm:id/n9e']")
logging.info(f"found {len(moment_node_list)} n9a / n9e node")
if len(moment_node_list) == 0:
    logging.error("no moment found!")
    

In [ ]:
for i, moment_node_i in enumerate(moment_node_list):
    left, high, right, low = AndroidUIHelper.get_box_from_bounds_raw_str(bounds=moment_node_i.attrib.get("bounds"))
    # logging.info("this_moment_node is with a high that is too small, skipping it.")
    logging.info(f"moment node {i}, high={high}, low={low}")
    if high > 192 and low < 1918:
        logging.info(f"choosing moment node {i}, with all the box visible")
        this_moment_low = low
        this_moment_node = moment_node_i
        break
    logging.warning("no suitable moment found, skipping")

In [ ]:
this_moment_tree = ET.ElementTree(this_moment_node)
this_moment_tree.write(temp_data_base_dir + "/this_moment_tree.xml")

In [ ]:
for ele in this_moment_tree.iter():
    print(ele, ele.attrib)

In [ ]:
for i, node in enumerate(this_moment_tree.iter()):
    resource_id = node.attrib.get("resource-id", "no-resource-id")
    content_desc = node.attrib.get("content-desc", "no-content-desc")
    text = node.attrib.get("text", "no-text")
    left, high, right, low = AndroidUIHelper.get_box_from_bounds_raw_str(bounds=node.attrib.get("bounds"))
    logging.debug(f"itering within child nodes, {i}, {content_desc}, {text}, {left}, {high}, {right}, {low}")
    if resource_id == "com.tencent.mm:id/n9a":
        logging.debug(f"skipping this node {i}, it's a n9a, 朋友圈主node")
    elif resource_id == "com.tencent.mm:id/n9e":
        logging.debug(f"skipping this node {i}, it's a n9e, 朋友圈主node")
    elif resource_id == "com.tencent.mm:id/od":
        logging.debug(f"skipping this node {i}, it's a od, 头像")
    elif resource_id == "com.tencent.mm:id/n96":
        n96 = text
        if len(n96) > 0:
            # TODO: 被折叠的朋友圈
            print(n96)
            logging.info(f"found n96, 被折叠的朋友圈 is {n96}")
        else:
            logging.debug(f"skipping this node {i}, it's a n96 and length is 0, 被折叠的朋友圈")
    elif resource_id == "com.tencent.mm:id/kbq":
        kbq = text
        # TODO: 保存用户
        print(kbq)
        logging.info(f"found kbq, user is {kbq}")
    elif resource_id == "com.tencent.mm:id/h9t":
        logging.debug(f"skipping this node {i}, it's a h9t, odj的父节点")
    elif resource_id == "com.tencent.mm:id/odj":
        odj = text
        # TODO: 保存分享链接标题
        print(odj)
        logging.info(f"found odj, 分享链接标题 is {odj}")
    elif resource_id == "com.tencent.mm:id/cuf" or resource_id == "com.tencent.mm:id/cut":
        cuf_cut = text
        if len(cuf_cut) > 0:
            # TODO: 保存文字
            print(cuf_cut)
            logging.info(f"found cuf_cut, 朋友圈内容文字 is {cuf_cut}")
        else:
            logging.debug(f"skipping this node {i}, it's a cuf / cut and length is 0")
    elif "图片" in content_desc:
        # TODO: 保存图片
        print(left, right, high, low)
        logging.info(f"found picture, left={left}, right={right}, high={high}, low={low}")
    elif resource_id == "com.tencent.mm:id/r2":
        logging.debug(f"skipping this node {i}, it's a r2, 评论区图标")
    elif resource_id == "com.tencent.mm:id/hbr":
        logging.debug(f"skipping this node {i}, it's a hbr, 点赞区图标")
    elif resource_id == "com.tencent.mm:id/n9v":
        n9v = text
        # TODO: 保存点赞的人
        print(n9v)
        logging.info(f"found cut, 点赞的人 is {n9v}")
    elif resource_id == "com.tencent.mm:id/c6p":
        c6p = text
        # TODO: 保存此条评论
        print(c6p)
        logging.info(f"found c6p, 此条评论 is {c6p}")
    elif text is not None and len(text) > 0:
        if node.attrib.get("clickable", "false") == "true":
            # TODO: 保存地址?
            logging.info(f"found extra text, maybe location, text is {text}")
        else:
            # TODO: 保存日期?
            logging.info(f"found extra text, maybe a datetime or location, text is {text}")
    else:
        logging.debug(f"ignoring this node, {i}, {node.attrib}")

In [ ]:
# this_avatar_node = this_moment_node.find(".//*[@resource-id='com.tencent.mm:id/od']")
this_avatar_node = this_moment_node.find("*[@resource-id='com.tencent.mm:id/od']")


In [ ]:
# bounds = this_avatar_node.attrib.get("bounds", "bounds")
# bounds = bounds.replace('[', '').replace(']', ',').split(',')
# bounds = map(int, bounds[:4])
left, high, right, low = AndroidUIHelper.get_box_from_bounds_raw_str(bounds=this_avatar_node.attrib.get("bounds"))


In [ ]:
this_avatar_image = cv2.imread(screenshot_raw_path)[high:low, left:right]

In [ ]:
cv2.imwrite(temp_data_base_dir + "/this_avatar_image.png", this_avatar_image)

In [ ]:
from IPython.display import display, Image
display(Image(temp_data_base_dir + "/this_avatar_image.png"))

In [ ]:
next_swipe_up_distance = this_moment_low - 300
logging.info(f"finished this moment, moving to the next one, next_swipe_up_distance={next_swipe_up_distance}")